In [1]:
#get the latest version of pyFlammarion
 
!pip install git+https://github.com/RecognitionAnalytics/pyFlammarion.git

import pyFlammarion.ImagingTools as afm 
import matplotlib.pyplot as plt 

  Running command git clone --filter=blob:none --quiet https://github.com/RecognitionAnalytics/pyFlammarion.git 'C:\Users\bashc\AppData\Local\Temp\pip-req-build-hej80l_k'


  Cloning https://github.com/RecognitionAnalytics/pyFlammarion.git to c:\users\bashc\appdata\local\temp\pip-req-build-hej80l_k
  Resolved https://github.com/RecognitionAnalytics/pyFlammarion.git to commit 8c51a067c70104478ed89888477cacf6721a40a9
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pyFlammarion: filename=pyflammarion-2025.5.21.post18-py3-none-any.whl size=5245723 sha256=b0a8511d7f957ec266a954961912d17c1de140b3d759acdf1f6d2004674b7da7
  Stored in directory: C:\Users\bashc\AppData\Local\Temp\pip-ephem-wheel-cache-snbdro9i\wheels\72\24\97\77bbf2b128db5c9fd0bd72bfa861f522377162d084fbb1c104
Successfully built pyFlammarion
  Attempting uninstall: pyFlammarion
    Found existing instal

ModuleNotFoundError: No module named 'FileLoaders'